# The Email Assistant Agent

> Note: This notebook demonstrates the final agent. See `langgraph_101.ipynb` for a simpler introduction to LangGraph concepts.

This notebook shows how to run our email assistant, which combines multiple features we've built: a triage router, a response agent with tools, human-in-the-loop (HITL) capabilities, and a persistent reminder system.

![overview-img](img/overview.png)

### 1. Load Environment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file in the parent directory
load_dotenv("../.env")

# Set to 1 to auto-accept any human-in-the-loop prompts
os.environ["HITL_AUTO_ACCEPT"] = "1"

### 2. Import and Run the Agent

All of our agent logic has been modularized into the `src/email_assistant` directory. We can now import the final, compiled agent directly.

In [ ]:
import sys
import uuid
# Add src to path to allow for imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from email_assistant.email_assistant_hitl_memory_gmail import email_assistant

In [ ]:
# Here is an example of an email requiring a response
email_input = {
  "id": "thread_abc_123",
  "from": "Alice <alice@example.com>",
  "to": "me@example.com",
  "subject": "Quick question about API documentation",
  "body": "Hi,\nI was reviewing the API documentation and had a question about the auth endpoint.\nThanks!\nAlice"
}

# The config is necessary to provide a unique thread_id for the checkpointer
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# Run the agent
response = email_assistant.invoke({"email_input": email_input}, config)

print(response.get('messages', [])[-1].content)

### 3. Advanced Feature: Reminders & Follow-ups

We have added a persistent reminder system to the agent. Here is a demonstration of the end-to-end workflow.

#### Step A: Clean the Database

First, let's ensure the reminder database is empty so we start from a clean state.

In [ ]:
db_path = os.getenv("REMINDER_DB_PATH", "../.local/reminders.db")
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Removed existing database at {db_path}")

#### Step B: Create a Reminder

We'll process an important email. The agent should classify it as `respond` or `notify` and create a reminder in the database.

In [ ]:
import json

with open('../tests/evaluation_data/reminders/01_create_reminder.txt', 'r') as f:
    create_email_data = json.load(f)

config = {"configurable": {"thread_id": str(uuid.uuid4())}}
email_assistant.invoke({"email_input": create_email_data}, config)

#### Step C: Verify Reminder Creation

We can now use the `reminder_worker.py` script's `--list` command to see the reminder that was just created. We use the `!` prefix to run shell commands from the notebook.

In [ ]:
!python ../scripts/reminder_worker.py --list

#### Step D: Cancel the Reminder

Next, we process a reply from the user in the same thread. The agent should detect this reply and automatically cancel the pending reminder.

In [ ]:
with open('../tests/evaluation_data/reminders/02_cancel_reminder.txt', 'r') as f:
    cancel_email_data = json.load(f)

config = {"configurable": {"thread_id": str(uuid.uuid4())}}
email_assistant.invoke({"email_input": cancel_email_data}, config)

#### Step E: Verify Reminder Cancellation

Finally, we list the reminders again. The list should now be empty.

In [ ]:
!python ../scripts/reminder_worker.py --list